# QLD Data Pipeline

## Install and load Packages

### Prerequisites

- A correct version of chromedriver is required on the working directory. \
Please install the version that matches your browser.
https://chromedriver.chromium.org/downloads

- Ghostscript and ActiveTcl are required to be installed on the machine. \
See instructions on:
https://camelot-py.readthedocs.io/en/master/user/install-deps.html \
Re-boot maybe required following a new installation.

### Load Common Libraries

In [19]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
try:
    import scrapy # scrape webpage
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
# text cleaning
import re
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
print("Installed Python version" , platform.python_version())

Installed Python version 3.8.8


## Extract & Transform
Process all files into CSV and load CSVs

### Extract Lobbyist Data

The Register of Lobbyists is a list of professional lobbyists who wish to lobby Government representatives. \
Extracting the QLD lobbysit data from https://lobbyists.integrity.qld.gov.au/register-details/list-companies.aspx

#### middlewares

In [ ]:
from scrapy import signals

# useful for handling different item types with a single interface
from itemadapter import is_item, ItemAdapter


class QldlobbyistSpiderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the spider middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_spider_input(self, response, spider):
        # Called for each response that goes through the spider
        # middleware and into the spider.

        # Should return None or raise an exception.
        return None

    def process_spider_output(self, response, result, spider):
        # Called with the results returned from the Spider, after
        # it has processed the response.

        # Must return an iterable of Request, or item objects.
        for i in result:
            yield i

    def process_spider_exception(self, response, exception, spider):
        # Called when a spider or process_spider_input() method
        # (from other spider middleware) raises an exception.

        # Should return either None or an iterable of Request or item objects.
        pass

    def process_start_requests(self, start_requests, spider):
        # Called with the start requests of the spider, and works
        # similarly to the process_spider_output() method, except
        # that it doesn’t have a response associated.

        # Must return only requests (not items).
        for r in start_requests:
            yield r

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)


class QldlobbyistDownloaderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the downloader middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_request(self, request, spider):
        # Called for each request that goes through the downloader
        # middleware.

        # Must either:
        # - return None: continue processing this request
        # - or return a Response object
        # - or return a Request object
        # - or raise IgnoreRequest: process_exception() methods of
        #   installed downloader middleware will be called
        return None

    def process_response(self, request, response, spider):
        # Called with the response returned from the downloader.

        # Must either;
        # - return a Response object
        # - return a Request object
        # - or raise IgnoreRequest
        return response

    def process_exception(self, request, exception, spider):
        # Called when a download handler or a process_request()
        # (from other downloader middleware) raises an exception.

        # Must either:
        # - return None: continue processing this exception
        # - return a Response object: stops process_exception() chain
        # - return a Request object: stops process_exception() chain
        pass

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)

#### setup a pipeline

In [ ]:
from itemadapter import ItemAdapter

class QldlobbyistPipeline:
    def process_item(self, item, spider):
        return item

#### model for scraped items

In [ ]:
class QldlobbyistItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


#### Define the spider

In [ ]:
from twisted.internet import reactor
import scrapy
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging

# Reactor restart
from crochet import setup, wait_for
setup()

class QldlobbySpider(scrapy.Spider):
    name = 'qldlobbyist'
    allowed_domains = ['lobbyists.integrity.qld.gov.au']
    start_urls = ['https://lobbyists.integrity.qld.gov.au/register-details/list-lobbyists.aspx']
    custom_settings = {
        'DOWNLOAD_DELAY': '0',
        'BOT_NAME': 'qldlobbyist',
        'SPIDER_MODULES': 'qldlobbyist.spiders',
        'NEWSPIDER_MODULE': 'qldlobbyist.spiders',
        'ROBOTSTXT_OBEY': 'False',
        'FEEDS': {
            'data/qldlobbyist.csv': { # csv output
                'format': 'csv',
                'overwrite': True
            }
        }
    }
    def parse(self, response):
            url =[]
            endings = response.xpath('//*[@id="ListView"]/li/a/@href')
            for ending in endings:
                url.append('https://lobbyists.integrity.qld.gov.au/register-details/'+ending.get())

            for u in url:
                yield scrapy.Request(url=u, callback = self.parse_client_data)
    
    def parse_client_data(self, response):
        yield {
            'Lobbyist Name' : response.xpath('//*[@id="ctl00_ContentPlaceholder1_lblName"]/text()').get(),
            'Lobbying Firm' : response.xpath('//*[@id="article"]/div/table/tr[2]/td[2]//text()').get(),
            'ABN' : response.xpath('//*[@id="article"]/div/table/tr[3]/td[2]//text()').get(),
            'Position' : response.xpath('//*[@id="article"]/div/table/tr[4]/td[2]//text()').get(),
            }
        
def run_spider():
    """run spider with qldlobbyist"""
    crawler = CrawlerProcess()
    d = crawler.crawl(QldlobbySpider)
    return d

#### Start the crawler

In [ ]:
run_spider()

#### Read the output

In [ ]:
lobbyist = pd.read_csv('data/qldlobbyist.csv')
lobbyist.head()

### Extract Lobbyist Client Data

This data consists of third party clients of the Lobbyist that currently retain the services of the business to provide paid or unpaid lobbyist services. \
Extracting the QLD lobbysit clients data from https://lobbyists.integrity.qld.gov.au/register-details/list-clients.aspx

####  The process for scraping the lobbyist client data uses the existing settings, and classes from the lobbyist data scraping process

#### Define the spider

In [ ]:
from scrapy.crawler import CrawlerProcess

# Reactor restart
from crochet import setup, wait_for
setup()

class QldlobClientSpider(scrapy.Spider):
    name = 'qldlobbyclient'
    allowed_domains = ['lobbyists.integrity.qld.gov.au']
    start_urls = ['https://lobbyists.integrity.qld.gov.au/register-details/list-clients.aspx']
    custom_settings = {
                'DOWNLOAD_DELAY': '0',
        'BOT_NAME': 'qldlobbyist',
        'SPIDER_MODULES': 'qldlobbyist.spiders',
        'NEWSPIDER_MODULE': 'qldlobbyist.spiders',
        'ROBOTSTXT_OBEY': 'False',
        'FEEDS': {
            'data/qldlbClient.csv': { # csv output
                'format': 'csv',
                'overwrite': True
            }
        }
    }
    def parse(self, response):
            url =[]
            endings = response.xpath('//*[@id="ListView"]/li/a/@href')
            for ending in endings:
                url.append('https://lobbyists.integrity.qld.gov.au/register-details/'+ending.get())

            for u in url:
                yield scrapy.Request(url=u, callback = self.parse_client_data)
    
    def parse_client_data(self, response):
        yield {
            'Client Name' : response.xpath('//*[@id="ctl00_ContentPlaceholder1_lblCName"]/text()').get(),
            'Lobbyist' : response.xpath('//*[@id="article"]/div/table/tr[2]/td[2]//text()').get(),
            'ABN' : response.xpath('//*[@id="article"]/div/table/tr[3]/td[2]//text()').get(),
            }

def run_spider():
    """run spider with qldlobbyclient"""
    crawler = CrawlerProcess()
    d = crawler.crawl(QldlobClientSpider)
    return d

#### Start the crawler

In [ ]:
run_spider()

#### Read the output

In [ ]:
client = pd.read_csv('data/qldlbClient.csv')
client.head()

### Extract Donations Data

A disclosure return is the reporting of all donations, loans, and expenditure incurred for an election campaign. 
These must be reported to the ECQ under the Electoral Act 1992 and the Local Government Electoral Act 2011. 
All disclosures are made through the Electronic Disclosure System (EDS) and are available to the public.

This data is scraped from the EDS (https://disclosures.ecq.qld.gov.au/Map)

In [ ]:
# Import Packages
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
import datetime

# Define the browser to use and then open up the donations homepage
driver = webdriver.Chrome()
driver.get('https://disclosures.ecq.qld.gov.au/Map')

# Empty List to collect new urls
recipient_url = []

# Read the latest donations csv and determine latest date
df_date = pd.DataFrame()
df_donations = pd.read_csv('data/qld_donations.csv')
df_date['date'] = df_donations.date.dropna()
df_date['date'] = pd.to_datetime(df_date['date'], format='%d/%m/%Y')
last_date = max(df_date.date)
newest_date = (last_date.date().strftime('%d-%m-%Y'))

# Input the last date into the website, then apply filter
date_from = driver.find_element(By.XPATH, "//*[@id='ViewFilter_DateFrom']")
date_from.send_keys(newest_date)
# date_from.send_keys('01-09-2022') # for manual filter
apply_button = driver.find_element(By.XPATH, "//*[@id='maps-form']/div/div/div[2]/div[1]/button")
apply_button.click()
time.sleep(2)

# 2. Begin scraping the urls
while True:
    
     # This code selects all the URLs from the main table, then appends URLs to a list
    recip_urls = driver.find_elements(By.XPATH, "//tr/td[3]/a")
    for url in recip_urls:
        #print(url.get_attribute("href"))
        recipient_url.append(url.get_attribute("href"))
    
    # for current page, grab the page items at bottom
    page_items_element = driver.find_element(By.XPATH, "//*[@id='map-page-header']/div[2]/div[3]/div/div/div/div/div/div[2]/small")
    page_items = page_items_element.text
    page_item_nos = re.findall(r'\d+', page_items)
    
    # pagination if page items are not at the end
    if page_item_nos[1] != page_item_nos[2]:
        # looks for the next button
        button = driver.find_element(By.CLASS_NAME, "fa.fa-chevron-right")
        button.click()
        time.sleep(2)
    
    else:
        # exits when the button no longer exists
        break

# Instantiate all lists        
all_urls = recipient_url
date = []
donor_name = []
donor_type = []
recipient_name = []
recipient_type = []
agent_names = []
gift_type = []
gift_value = []
page_url= []


# Scrape the pages from each of the just scraped urls
for url in all_urls:
    driver.get(url)
    
    # Donor
    try: 
        driver.find_element(By.XPATH, "//*[@class='form-group']/div/input")
    except NoSuchElementException:
        donor_name.append(None)
    else:
        donor_dim = driver.find_element(By.XPATH, "//*[@class='form-group']/div/input")
        donor_name.append(donor_dim.get_attribute("value"))
    
    
    # Donor_Types
    try: 
        driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[4]")
    except NoSuchElementException:
        donor_type.append(None)
    else:
        donor_type_dim = driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[4]")
        donor_type.append(donor_type_dim.text)

    # Recipient
    try:
        driver.find_element(By.XPATH, "//*[@id='Head_ElectorFullName']")
    except NoSuchElementException:
        try:
            driver.find_element(By.XPATH, "//*[@id='Head_PoliticalPartyTitle']")
        except NoSuchElementException:
            try:
                driver.find_element(By.XPATH, "//*[@id='Head_Title']")
            except NoSuchElementException:
                recipient_name.append(None)
            else:
                recipient_dim = driver.find_element(By.XPATH, "//*[@id='Head_Title']")
                recipient_name.append(recipient_dim.get_attribute("value"))
        else:
            recipient_dim = driver.find_element(By.XPATH, "//*[@id='Head_PoliticalPartyTitle']")
            recipient_name.append(recipient_dim.get_attribute("value"))
    else:
        recipient_dim = driver.find_element(By.XPATH, "//*[@id='Head_ElectorFullName']")
        recipient_name.append(recipient_dim.get_attribute("value"))    
    
    
    
    # Recipient Type
    try:
        driver.find_element(By.XPATH, "//*[@id='content']/div/div[1]/div/div/div/div/div[1]/h1")
    except NoSuchElementException:
        recipient_type.append(None)
    else:
        recip_type_dim = driver.find_element(By.XPATH, "//*[@id='content']/div/div[1]/div/div/div/div/div[1]/h1")
        recipient_type.append(recip_type_dim.text)
    
    # Date
    try:
        driver.find_element(By.XPATH, "//*[@class='form-control datepicker gift-date-received special-reporting-event-check-trigger']")
    except NoSuchElementException:
        date.append(None)
    else:
        date_dim = driver.find_element(By.XPATH, "//*[@class='form-control datepicker gift-date-received special-reporting-event-check-trigger']")
        date.append(date_dim.get_attribute("value"))
    
    # Gift value
    try:
        driver.find_element(By.XPATH, "//*[@class='form-control currencyFormat text-right gift-amount special-reporting-event-check-trigger']")
    except NoSuchElementException:
        gift_value.append(None)
    else:
        gift_value_dim = driver.find_element(By.XPATH, "//*[@class='form-control currencyFormat text-right gift-amount special-reporting-event-check-trigger']")
        gift_value.append(gift_value_dim.get_attribute("value"))
    
    # Gift Type
    try:
        driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[2]")
    except NoSuchElementException:
        gift_type.append(None)
    else:
        gift_type_dim = driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[2]")
        gift_type.append(gift_type_dim.text)
    
    # Agent Names
    try:
        driver.find_element(By.XPATH, "//*[@id='Head_RepresentativeFullName']")
    except NoSuchElementException:
        try: 
            driver.find_element(By.XPATH, "//*[@id='Head_AgentFullName']")
        except NoSuchElementException:
            agent_names.append(None)
        else:
            agent_name = driver.find_element(By.XPATH, "//*[@id='Head_AgentFullName']")
            agent_names.append(agent_name.get_attribute("value"))   
    else:
        agent_name = driver.find_element(By.XPATH, "//*[@id='Head_RepresentativeFullName']")
        agent_names.append(agent_name.get_attribute("value"))
        
        
    # original url
    page_url.append(driver.current_url)
        
    time.sleep(0.5)
    
    
# Create dataframe for the newly scraped pages
df = pd.DataFrame(list(zip(date, donor_name, donor_type, recipient_name, recipient_type, agent_names, gift_type, gift_value, page_url)), 
                  columns = ["date", "donor_name", "donor_type", "recipient_name", "recipient_type", "agent_names", "gift_type", "gift_value", "page_url"])

# Clean any whitespaces
df['recipient_name'] = df['recipient_name'].str.strip()
df['donor_name'] = df['donor_name'].str.strip()
df['agent_names'] = df['agent_names'].str.strip()

# Import existing data, concatenate the new data with existing dataset, remove duplicates
df_concat = pd.concat([df, df_donations], ignore_index=True)
df_dupes_dropped = df_concat.drop_duplicates(subset=['page_url'])

# Save new file
df_dupes_dropped.to_csv('data/qld_donations.csv', header=True, index=False)

# close the browser
driver.quit()

In [ ]:
donations = df_dupes_dropped
donations.head()

### Extract Ministerial Diaries Data

Ministers are required to proactively disclose on a monthly basis portfolio related meetings and events.

For any meeting with a registered lobbyist or any person working for the lobbyist in any capacity, other than administrative staff, the diary must also include details about all attendees and a short description of the subject matter of the meeting.

Personal, electorate or party political meetings or events, media events and interviews and information contrary to public interest (e.g. meetings regarding sensitive law enforcement, public safety or whistle-blower matters) are not to be released.

This data is scraped from https://cabinet.qld.gov.au/ministers-portfolios.aspx

In [14]:
from bs4 import BeautifulSoup as bs
import requests
import camelot
import glob

In [3]:
import ctypes
from ctypes.util import find_library

In [4]:
# This checks if Ghostscript is installed correctly. This shouldn't return blank.
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

'C:\\Program Files\\gs\\gs10.00.0\\bin\\gsdll64.dll'

In [5]:
# set domain
QLD = "https://cabinet.qld.gov.au/ministers-portfolios.aspx"

In [ ]:
# scrape links for ministrial diaries (pdf download links)
page = requests.get(QLD)
soup = bs(page.content, "html.parser")
results = soup.find_all("a")

urls = []
names = []
error_files = []
d = {}
for i, link in enumerate(results):
    current_link = link.get('href')
    if 'ministers-portfolios/' in current_link:
        
        current_link = current_link.split("/")[-1].split(".")[0]
        #print(current_link)
        
       
        minister_page = requests.get("https://cabinet.qld.gov.au/ministers-portfolios/"+current_link+".aspx")
        
        minister_soup = bs(minister_page.content, "html.parser")
        minister_results = minister_soup.find_all("a")
            
        
        for l, minister_link in enumerate(minister_results):
            current_link1 = minister_link.get('href')
            
            if current_link1.endswith('pdf'):
                current_link1 = current_link1.replace("/ministers-portfolios/", "")
                urls.append(current_link1)
                names.append(current_link)
            


In [ ]:
# store download pdfs
names_urls = zip(names, urls)

errorURL= []

for name, url in names_urls:
    if "http://" not in url:
        try: 
            if "charter-letter" not in url:
                year, month = url.split("/")[2], url.split("/")[3]
            else:
                month = "charter"
                year = "-letter"
            r = requests.get("https://cabinet.qld.gov.au/ministers-portfolios/"+url)
            with open("qld_min_diaries//" + name+"_"+month+year+".pdf", "wb") as f:
                f.write(r.content)
        except: 
            errorURL.append(url)
            pass
    

#### Extract from pdf

##### Function to get minister's name from the file name

In [39]:
def get_portfolio(file_name):
    Portfolio_temp =file_name.split("_") # changed from - to _ to get full names
    cmonth =  ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    Portfolio = ""
    for prt in Portfolio_temp:
        if isinstance(prt, int) or prt in cmonth or "pdf" in prt:
            pass
        else:
            Portfolio += prt +" "
    return Portfolio

##### Function to get table from the pdf

In [40]:
def extract_tables(file, portfolio):
    tables = camelot.read_pdf(file, pages='all')

    for table in tables:
        for idx, row in table.df.iterrows():
            if row[0] == "":
                date = next_date
            else:
                date = row[0]
            
            
            purpose = row[2]
            if purpose == "":
                    purpose = next_purpose  #if the next page does not have date or purpose take it from previous page
                    
            if "Date" not in row[0]:
                orgs = row[1].split("\n")
                next_date = date
                next_purpose = purpose #if the next page does not have date or purpose take it from previous page
                for org in orgs:
                    org = org.replace("¬†", " ")
                    df["Date"].append(date)
                    df["Ministers"].append(portfolio)
                    df["Organisation/Individual"].append(org)
                    df["Purpose of Meeting"].append(purpose)
            

In [41]:
diaries = { "Date":[], "Ministers":[], "Organisation/Individual": [], "Purpose of Meeting":[]}
# exclude letter.pdf
pdf_files = list(set(glob.glob("qld_min_diaries/*.pdf"))-set(glob.glob("qld_min_diaries/*letter.pdf")))
err_files = []
for file in pdf_files:
    file_name = file[len("qld_min_diaries/"):]
    portfolio = get_portfolio(file_name)
    try:
        extract_tables(file, portfolio)
    except:
        err_files.append(file_name)
        pass

diaries = pd.DataFrame(diaries)
diaries.head() 

,Date,Ministers,Organisation/Individual,Purpose of Meeting


In [16]:
diaries.to_csv("data/qld_diaries.csv", index=False)

# Pre-processing
To extract 'source', 'target', and 'weight' columns

In [ ]:
# lobbyist
ag_lobbyist = lobbyist.groupby(['Lobbying Firm','Lobbyist Name']).size().reset_index(name='weight')
ag_lobbyist = ag_lobbyist.rename(columns={'Lobbying Firm': 'source', 'Lobbyist Name': 'target'})
ag_lobbyist['color'] = '#9b5de5'
ag_lobbyist['group'] = 'Lobbyist'
ag_lobbyist.head()

In [ ]:
# client
ag_client = client.groupby(['Lobbyist','Client Name']).size().reset_index(name='weight')
ag_client = ag_client.rename(columns={'Lobbyist': 'source', 'Client Name': 'target'})
ag_client['color'] = '#fee440'
ag_client['group'] = 'Client'
ag_client.head()

In [ ]:
# donations
ag_donations = donations.groupby(['Donor','Recipient']).size().reset_index(name='weight')
ag_donations = ag_donations.rename(columns={'Donor': 'source', 'Recipient': 'target'})
ag_donations['color'] = '#00bbf9'
ag_donations['group'] = 'Donnation'
ag_donations.head()

In [ ]:
# donations - with amount
ag_donations_sum = donations.groupby(['Donor','Recipient'])['Gift value'].agg(['sum', 'count']).reset_index()
ag_donations_sum = ag_donations_sum.rename(columns={'Donor': 'source', 'Recipient': 'target', 'sum':'amount', 'count':'weight'})
ag_donations_sum.head()

In [ ]:
# diaries
# client
ag_diaries = diaries.groupby(['Party','Name']).size().reset_index(name='weight')
ag_diaries = ag_diaries.rename(columns={'Party': 'source', 'Name': 'target'})
ag_diaries['color'] = '#00f5d4'
ag_diaries['group'] = 'Ministers'
ag_diaries.head()

# Stack all data sets
map_dat

In [ ]:
map_dat = pd.concat([ag_lobbyist,ag_client,ag_donations,ag_diaries],ignore_index=True)
#map_dat.to_csv(r'data\map_dat.csv', encoding='utf-8', index=False)

map_dat.head()

## Different Algorithms for Network
barnes_hut: based on vectorisation


In [ ]:
def map_algs(g, alg="barnes"): #setting default algorithm to barnes_hut
    if alg=="barnes":
        g.barnes_hut()
    if alg=="forced":
        g.forced_atlas_2based()
    if alg=="hrepulsion":
        g.hrepulsion()

## Available shapes for nodes:
 LABEL INSIDE: ellipse, cirlce, database, box, text
 LABEL OUTSIDE: image, circularImage, diamond, dot, star, triangle, triangleDown, square, icon
 size(num(optional)), only for nodes with outside labels

In [ ]:
def map_data(map_dat, 
            src_color="#03DAC6", tgt_color="#da03b3", edge_color="#018786",
            src_shape="cirlce", tgt_shape="cirlce",
            algorithm="barnes",
            buttons=False):
    g = Network(height = '1000px', width = '100%', bgcolor='#222222', font_color='white', directed=False)
    
    #create a node for each donor and recipient
    if buttons==True:
        g.width="75%"
        g.show_buttons(filter_=["physics", "edges", "nodes", "layout"])
        # physics button might be most useful for users, real time modification of node behaviour
        # the different button panels are nodes, layout, interaction, manipulation, selection, renderer, physics
        
    #Dynamic edges can make it so we can see multiple interactions between nodes
        #width and smooth options some of the most useful ones
    
    #Changing node sizes, can only change those with lables on the outside.
        #Can change shadow, border, size, etc
        
    #Interaction panel:
        #Can enable navigation buttons
        #Multiselect options
        #Hover will allow you to highlight edges/nodes by hovering
        
    #Copy and paste javascript into HTML file with all of the specific options selected
    
    #set your data
    source = map_dat['source']
    target = map_dat['target']
    weight = map_dat['weight']
    edge_data = zip(source, target, weight)
    
    for e in edge_data:
        src = e[0]
        tgt = e[1]
        wgt = e[2]
        g.add_node(src, src, title=src, color=src_color, shape=src_shape)
        g.add_node(tgt, tgt, title=tgt, color=tgt_color, shape=tgt_shape)
        g.add_edge(src, tgt, value=wgt, color=edge_color)
    
    #sets algorithm
    map_algs(g, alg=algorithm)
    
    #setting the edges as dynamic
    g.set_edge_smooth("dynamic") #should see MULTIPLE facets of edges between nodes. can make different edges different colours
    g.show("map_network_v1.0.html")

In [ ]:
map_data(map_dat, buttons=True)